In [3]:
# 選擇權參數

S0 = 100
K = 105
T = 1
r = 0.05
Sigma = 0.2

# Options Pricing via Monte Carlo Simulation

In [27]:
import numpy as np

N = 100000    # simulation trials

z = np.random.standard_normal(N)
ST = S0 * np.exp((r - 0.5 * Sigma ** 2) * T + Sigma * np.sqrt(T) * z)
CT = np.maximum(ST - K, 0)
C0 = np.exp(-r * T) * np.sum(CT) / N

C0

8.0088202343802521

In [21]:
print('The present value of the European call option: %5.3f' % C0)

The present value of the European call option: 8.083


In [22]:
def Call_Value_MSC(S0, K, T, r, Sigma, N):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    N: simulation trials
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from numpy import random, sqrt, exp, maximum, sum
    
    z = random.standard_normal(N)
    ST = S0 * exp((r - 0.5 * Sigma ** 2) * T + Sigma * sqrt(T) * z)
    CT = maximum(ST - K, 0)
    C0 = exp(-r * T) * sum(CT) / N
    return C0

In [25]:
Call_Value = Call_Value_MSC(S0, K, T, r, Sigma, N)
Call_Value

7.9597202836333416

# Options Pricing via BSM Model

In [8]:
def Call_Value_BSM(S0, K, T, r, Sigma):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from math import log, sqrt, exp
    from scipy import stats
    
    d1 = (log(S0 / K) + (r + 0.5 * Sigma ** 2) * T) / (Sigma * sqrt(T))
    d2 = (log(S0 / K) + (r - 0.5 * Sigma ** 2) * T) / (Sigma * sqrt(T))
    value = (S0 * stats.norm.cdf(d1, 0, 1)) - K * exp(-r * T) * stats.norm.cdf(d2, 0, 1)
    return value

In [14]:
Call_Value = Call_Value_BSM(S0, K, T, r, Sigma)
Call_Value

8.0213522351431763

In [15]:
print('The present value of the European call option: %5.3f' % Call_Value)

The present value of the European call option: 8.021


In [4]:
# function call function
def BSM_Call(S0, K, T, r, Sigma):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from math import log, sqrt, exp
    from scipy import stats
    
    d1 = BSM_d1(S0, K, T, r, Sigma)
    d2 = BSM_d2(S0, K, T, r, Sigma)
    value = (S0 * stats.norm.cdf(d1, 0, 1)) - K * exp(-r * T) * stats.norm.cdf(d2, 0, 1)
    return value

def BSM_d1(S0, K, T, r, Sigma):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from math import log, sqrt, exp
    return (log(S0 / K) + (r + 0.5 * Sigma ** 2) * T) / (Sigma * sqrt(T))

def BSM_d2(S0, K, T, r, Sigma):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from math import log, sqrt, exp
    return (log(S0 / K) + (r - 0.5 * Sigma ** 2) * T) / (Sigma * sqrt(T))

In [5]:
BSM_Call(S0, K, T, r, Sigma)

8.0213522351431763

In [1]:
class OptionPricingModel:
    def __init__(self,S0, K, T, Sigma, r, option):
        self.S0=S0
        self.K=K
        self.r=r
        self.T=T
        self.Sigma=Sigma
        self.N=0
        self.option=option
    def BS_d1(self):
        from math import log,sqrt,exp
        from scipy import stats
        d1=(log(self.S0/self.K)+(self.r+0.5*self.Sigma*self.Sigma)*self.T)/(self.Sigma*sqrt(self.T))
        return d1
    def BS_d2(self):
        from math import log,sqrt,exp
        from scipy import stats
        d2=(log(self.S0/self.K)+(self.r+0.5*self.Sigma*self.Sigma)*self.T)/(self.Sigma*sqrt(self.T))-self.Sigma*sqrt(self.T)
        return d2
    def BS_CloseForm(self):        
        from math import log,sqrt,exp
        from scipy import stats
        if self.option==0:
            C0=self.S0*stats.norm.cdf(self.BS_d1(),0,1)-exp(-self.r*self.T)*self.K*stats.norm.cdf(self.BS_d2(),0,1) 
            option='call:'+str(C0)
        else:
            P0=exp(-self.r*self.T)*self.K*stats.norm.cdf(-self.BS_d2(),0,1)-self.S0*stats.norm.cdf(-self.BS_d1(),0,1)
            option='put:'+str(P0)
        self.display(option)       
    def BS_MSC(self,N=1000):   
        from numpy import random, sqrt, exp, maximum, sum  
        z = random.standard_normal(N)
        ST = self.S0 * exp((self.r - 0.5 * self.Sigma ** 2) * self.T + self.Sigma * sqrt(self.T) * z)
        if self.option==0:
            CT = maximum(ST - self.K, 0)
            C0 = exp(-r * T) * sum(CT) / N
            option='call:'+str(C0)
        else:
            CT = maximum(self.K - ST , 0)
            P0 = exp(-r * T) * sum(CT) / N
            option='put:'+str(P0)
        self.display(option) 
    def display(self,option):  
            print(option)

In [2]:
S0 = 100
K = 105
T = 1
r = 0.05
Sigma = 0.2
option = 0
N=10000
BS=OptionPricingModel(S0, K, T, Sigma,r,option)
BS.BS_MSC(N)

call:8.01141293438
